In [1]:
# import stuff
import os
import numpy as np
import torch
import torch.utils.data as data
from itertools import product as product
import time

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Function
import pandas as pd

# import dataset
from utils.dataset import VOCDataset, DatasetTransform, make_datapath_list, Anno_xml2list, od_collate_fn

# set up person only VOC dataset

In [2]:
# load files
vocpath = os.path.join("..", "VOCdevkit", "VOC2007")
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(vocpath, cls="person")

# make Dataset
voc_classes = ['person']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 256  # 画像のinputサイズを300×300にする

## DatasetTransformを適応
transform = DatasetTransform(input_size, color_mean)
transform_anno = Anno_xml2list(voc_classes)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase = "train", transform=transform, transform_anno = transform_anno)
val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DatasetTransform(
    input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=od_collate_fn, num_workers=8)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, collate_fn=od_collate_fn, num_workers=8)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

000009
000017
000021
000023
000030
000032
000035
000041
000048
000050
000051
000066
000073
000081
000083
000089
000101
000104
000110
000113
000125
000129
000131
000133
000138
000146
000150
000159
000162
000163
000164
000165
000169
000170
000171
000173
000174
000177
000190
000192
000193
000194
000200
000210
000218
000220
000222
000229
000232
000245
000251
000257
000259
000269
000275
000276
000278
000282
000285
000288
000298
000302
000305
000308
000320
000321
000322
000323
000328
000331
000337
000338
000352
000359
000367
000372
000374
000382
000394
000406
000407
000411
000419
000428
000433
000435
000438
000443
000446
000448
000463
000468
000470
000476
000477
000480
000482
000483
000498
000499
000500
000515
000516
000518
000520
000523
000524
000525
000526
000530
000531
000535
000541
000545
000554
000555
000579
000583
000589
000591
000597
000612
000613
000625
000626
000628
000633
000648
000654
000677
000684
000688
000690
000694
000695
000702
000709
000717
000726
000731
000733
000739
000742

In [3]:
# check operation
batch_iterator = iter(dataloaders_dict["train"])  # iter
images, targets = next(batch_iterator)  # get first element
print(images.size())  # torch.Size([4, 3, 300, 300])
print(len(targets))
print(targets[1].shape)  # check targets

torch.Size([32, 3, 256, 256])
32
torch.Size([1, 5])


In [4]:
targets[1]

tensor([[0.0633, 0.0000, 0.8167, 0.8119, 0.0000]])

# test with ssd model.

In [5]:
from utils.blazeface import SSD256

BlazeFace(
  (features): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): BlazeBlock(
      (conv1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (act): ReLU(inplace)
    )
    (4): BlazeBlock(
      (conv1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, a

In [6]:
# SSD300の設定
ssd_cfg = {
    'num_classes': 2,  # 背景クラスを含めた合計クラス数
    'input_size': 256,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [16, 8],  # 各sourceの画像サイズ
    'steps': [8, 16],  # DBOXの大きさを決める
    'min_sizes': [16, 32],  # DBOXの大きさを決める
    'max_sizes': [32, 100],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

net = SSD256(phase="train", cfg=ssd_cfg)

# SSDのweightsを設定

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

# set inits for loc and conf
net.loc.apply(weights_init)
net.conf.apply(weights_init)

# GPUが使えるか確認
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using:", device)

print("set weights!")

using: cuda:0
set weights!


In [7]:
print(net)

SSD256(
  (blaze): BlazeFace(
    (features): Sequential(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): BlazeBlock(
        (conv1): Sequential(
          (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))
          (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (act): ReLU(inplace)
      )
      (4): BlazeBlock(
        (conv1): Sequential(
          (0): Conv2d(24, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=24)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1))


In [8]:
from utils.ssd_model import MultiBoxLoss

# define loss
criterion = MultiBoxLoss(jaccard_thresh=0.5,neg_pos=3, device=device)

# optim
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)

In [9]:
def get_current_lr(epoch):
    lr = 1e-3
    for i,lr_decay_epoch in enumerate([120,180]):
        if epoch >= lr_decay_epoch:
            lr *= 0.1
    return lr

def adjust_learning_rate(optimizer, epoch):
    lr = get_current_lr(epoch)
    print("lr is:", lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [10]:
# モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("used device：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0  # epochの損失和
    epoch_val_loss = 0.0  # epochの損失和
    logs = []

    # epochのループ
    for epoch in range(num_epochs+1):
        
        adjust_learning_rate(optimizer, epoch)
        
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('train')
            else:
                if((epoch+1) % 5 == 0):
                    net.eval()   # モデルを検証モードに
                    print('-------------')
                    print('val')
                else:
                    # 検証は5回に1回だけ行う
                    continue

            # データローダーからminibatchずつ取り出すループ
            for images, targets in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                images = images.to(device)
                targets = [ann.to(device)
                           for ann in targets]  # リストの各要素のテンソルをGPUへ

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    # 順伝搬（forward）計算
                    outputs = net(images)

                    # 損失の計算
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()  # 勾配の計算

                        # 勾配が大きくなりすぎると計算が不安定になるので、clipで最大でも勾配2.0に留める
                        nn.utils.clip_grad_value_(
                            net.parameters(), clip_value=2.0)

                        optimizer.step()  # パラメータ更新

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('Iteration {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item(), duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 検証時
                    else:
                        epoch_val_loss += loss.item()

        # epochのphaseごとのlossと正解率
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

        # ログを保存
        log_epoch = {'epoch': epoch+1,
                     'train_loss': epoch_train_loss, 'val_loss': epoch_val_loss}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss = 0.0  # epochの損失和
        epoch_val_loss = 0.0  # epochの損失和

        # ネットワークを保存する
        if ((epoch+1) % 10 == 0):
            torch.save(net.state_dict(), 'weights/blazeface256_' +
                       str(epoch+1) + '.pth')

# start training here

In [11]:
num_epochs = 200
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

used device： cuda:0
lr is: 0.001
-------------
Epoch 1/200
-------------
train
Iteration 10 || Loss: 109.0224 || 10iter: 11.6945 sec.
Iteration 20 || Loss: 51.9891 || 10iter: 5.6860 sec.
Iteration 30 || Loss: 54.7372 || 10iter: 5.5014 sec.
Iteration 40 || Loss: 42.4690 || 10iter: 6.0593 sec.
Iteration 50 || Loss: 43.7510 || 10iter: 5.7763 sec.
Iteration 60 || Loss: 46.9673 || 10iter: 4.3901 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:3690.0698 ||Epoch_VAL_Loss:0.0000
timer:  42.5251 sec.
lr is: 0.001
-------------
Epoch 2/200
-------------
train
Iteration 70 || Loss: 42.3619 || 10iter: 8.0195 sec.
Iteration 80 || Loss: 48.6764 || 10iter: 5.8791 sec.
Iteration 90 || Loss: 42.2925 || 10iter: 5.4265 sec.
Iteration 100 || Loss: 49.2742 || 10iter: 5.6672 sec.
Iteration 110 || Loss: 35.3208 || 10iter: 5.4864 sec.
Iteration 120 || Loss: 39.1136 || 10iter: 4.6204 sec.
Iteration 130 || Loss: 33.9180 || 10iter: 5.3628 sec.
-------------
epoch 2 || Epoch_TRAIN_Loss:2603.7627 ||Epoch_VAL_Loss:0

Iteration 1060 || Loss: 19.5870 || 10iter: 5.0745 sec.
Iteration 1070 || Loss: 17.6985 || 10iter: 5.5093 sec.
Iteration 1080 || Loss: 15.0168 || 10iter: 4.5111 sec.
Iteration 1090 || Loss: 19.3302 || 10iter: 5.0082 sec.
Iteration 1100 || Loss: 16.6463 || 10iter: 6.9158 sec.
Iteration 1110 || Loss: 21.9359 || 10iter: 5.7515 sec.
Iteration 1120 || Loss: 18.5494 || 10iter: 4.4638 sec.
-------------
epoch 17 || Epoch_TRAIN_Loss:1223.7228 ||Epoch_VAL_Loss:0.0000
timer:  38.6584 sec.
lr is: 0.001
-------------
Epoch 18/200
-------------
train
Iteration 1130 || Loss: 17.0926 || 10iter: 8.8011 sec.
Iteration 1140 || Loss: 18.1224 || 10iter: 5.0017 sec.
Iteration 1150 || Loss: 12.5873 || 10iter: 6.7836 sec.
Iteration 1160 || Loss: 15.5221 || 10iter: 6.1109 sec.
Iteration 1170 || Loss: 21.5021 || 10iter: 6.0276 sec.
Iteration 1180 || Loss: 21.7728 || 10iter: 4.6665 sec.
-------------
epoch 18 || Epoch_TRAIN_Loss:1210.2345 ||Epoch_VAL_Loss:0.0000
timer:  41.1829 sec.
lr is: 0.001
-------------
Ep

Iteration 2110 || Loss: 13.9025 || 10iter: 4.7387 sec.
-------------
epoch 32 || Epoch_TRAIN_Loss:1096.9405 ||Epoch_VAL_Loss:0.0000
timer:  40.7990 sec.
lr is: 0.001
-------------
Epoch 33/200
-------------
train
Iteration 2120 || Loss: 13.5176 || 10iter: 9.1378 sec.
Iteration 2130 || Loss: 16.8139 || 10iter: 5.0485 sec.
Iteration 2140 || Loss: 11.9304 || 10iter: 4.9180 sec.
Iteration 2150 || Loss: 18.1645 || 10iter: 4.5516 sec.
Iteration 2160 || Loss: 18.4036 || 10iter: 4.6352 sec.
Iteration 2170 || Loss: 19.3296 || 10iter: 5.6246 sec.
-------------
epoch 33 || Epoch_TRAIN_Loss:1107.0035 ||Epoch_VAL_Loss:0.0000
timer:  38.1379 sec.
lr is: 0.001
-------------
Epoch 34/200
-------------
train
Iteration 2180 || Loss: 15.8720 || 10iter: 5.3103 sec.
Iteration 2190 || Loss: 14.3596 || 10iter: 7.2093 sec.
Iteration 2200 || Loss: 20.2769 || 10iter: 5.2394 sec.
Iteration 2210 || Loss: 15.8526 || 10iter: 4.6394 sec.
Iteration 2220 || Loss: 19.8023 || 10iter: 6.3191 sec.
Iteration 2230 || Loss: 

Iteration 3130 || Loss: 8.7473 || 10iter: 6.4241 sec.
Iteration 3140 || Loss: 19.3195 || 10iter: 5.7979 sec.
Iteration 3150 || Loss: 14.8356 || 10iter: 5.9334 sec.
Iteration 3160 || Loss: 16.5500 || 10iter: 4.9360 sec.
-------------
epoch 48 || Epoch_TRAIN_Loss:1066.0681 ||Epoch_VAL_Loss:0.0000
timer:  39.6614 sec.
lr is: 0.001
-------------
Epoch 49/200
-------------
train
Iteration 3170 || Loss: 10.7166 || 10iter: 3.3533 sec.
Iteration 3180 || Loss: 12.4312 || 10iter: 7.7737 sec.
Iteration 3190 || Loss: 14.7772 || 10iter: 5.3992 sec.
Iteration 3200 || Loss: 14.9387 || 10iter: 5.9460 sec.
Iteration 3210 || Loss: 15.3616 || 10iter: 5.9322 sec.
Iteration 3220 || Loss: 11.4271 || 10iter: 5.6583 sec.
Iteration 3230 || Loss: 17.5042 || 10iter: 4.1307 sec.
-------------
epoch 49 || Epoch_TRAIN_Loss:1018.1212 ||Epoch_VAL_Loss:0.0000
timer:  40.5144 sec.
lr is: 0.001
-------------
Epoch 50/200
-------------
train
Iteration 3240 || Loss: 17.9488 || 10iter: 6.4430 sec.
Iteration 3250 || Loss: 1

Iteration 4160 || Loss: 18.7167 || 10iter: 4.1566 sec.
Iteration 4170 || Loss: 14.2766 || 10iter: 7.3106 sec.
Iteration 4180 || Loss: 13.5269 || 10iter: 5.6517 sec.
Iteration 4190 || Loss: 17.2264 || 10iter: 5.8274 sec.
Iteration 4200 || Loss: 14.5326 || 10iter: 5.8990 sec.
Iteration 4210 || Loss: 11.4854 || 10iter: 5.0600 sec.
Iteration 4220 || Loss: 11.5764 || 10iter: 4.2474 sec.
-------------
epoch 64 || Epoch_TRAIN_Loss:990.7665 ||Epoch_VAL_Loss:0.0000
timer:  40.8916 sec.
lr is: 0.001
-------------
Epoch 65/200
-------------
train
Iteration 4230 || Loss: 14.2379 || 10iter: 8.8906 sec.
Iteration 4240 || Loss: 15.0938 || 10iter: 6.0149 sec.
Iteration 4250 || Loss: 18.1892 || 10iter: 5.9513 sec.
Iteration 4260 || Loss: 12.5868 || 10iter: 5.4881 sec.
Iteration 4270 || Loss: 18.6650 || 10iter: 4.6883 sec.
Iteration 4280 || Loss: 16.3001 || 10iter: 5.2455 sec.
Iteration 4290 || Loss: 16.4084 || 10iter: 4.8040 sec.
-------------
val
-------------
epoch 65 || Epoch_TRAIN_Loss:990.8403 ||E

...

-------------
epoch 126 || Epoch_TRAIN_Loss:912.7318 ||Epoch_VAL_Loss:0.0000
timer:  42.2085 sec.
lr is: 0.0001
-------------
Epoch 127/200
-------------
train
Iteration 8320 || Loss: 8.3842 || 10iter: 6.2899 sec.
Iteration 8330 || Loss: 11.3473 || 10iter: 6.5806 sec.
Iteration 8340 || Loss: 10.6595 || 10iter: 5.2735 sec.
Iteration 8350 || Loss: 21.6955 || 10iter: 4.7302 sec.
Iteration 8360 || Loss: 12.9487 || 10iter: 4.6884 sec.
Iteration 8370 || Loss: 14.2252 || 10iter: 4.0234 sec.
Iteration 8380 || Loss: 9.6251 || 10iter: 5.1787 sec.
-------------
epoch 127 || Epoch_TRAIN_Loss:891.2265 ||Epoch_VAL_Loss:0.0000
timer:  38.2336 sec.
lr is: 0.0001
-------------
Epoch 128/200
-------------
train
Iteration 8390 || Loss: 15.8108 || 10iter: 8.8776 sec.
Iteration 8400 || Loss: 12.3316 || 10iter: 5.8759 sec.
Iteration 8410 || Loss: 8.0750 || 10iter: 5.6826 sec.
Iteration 8420 || Loss: 11.4959 || 10iter: 4.9282 sec.
Iteration 8430 || Loss: 12.4636 || 10iter: 6.0788 sec.
Iteration 8440 || Loss:

Iteration 9340 || Loss: 10.2143 || 10iter: 7.2995 sec.
Iteration 9350 || Loss: 12.1264 || 10iter: 6.1153 sec.
Iteration 9360 || Loss: 15.4096 || 10iter: 6.0656 sec.
Iteration 9370 || Loss: 12.5955 || 10iter: 4.7365 sec.
-------------
epoch 142 || Epoch_TRAIN_Loss:923.2613 ||Epoch_VAL_Loss:0.0000
timer:  41.9464 sec.
lr is: 0.0001
-------------
Epoch 143/200
-------------
train
Iteration 9380 || Loss: 15.9039 || 10iter: 7.2000 sec.
Iteration 9390 || Loss: 11.7340 || 10iter: 6.9361 sec.
Iteration 9400 || Loss: 12.4938 || 10iter: 5.8139 sec.
Iteration 9410 || Loss: 13.3701 || 10iter: 5.6427 sec.
Iteration 9420 || Loss: 12.2007 || 10iter: 5.2970 sec.
Iteration 9430 || Loss: 13.1263 || 10iter: 4.8294 sec.
-------------
epoch 143 || Epoch_TRAIN_Loss:899.3416 ||Epoch_VAL_Loss:0.0000
timer:  39.4516 sec.
lr is: 0.0001
-------------
Epoch 144/200
-------------
train
Iteration 9440 || Loss: 12.0305 || 10iter: 4.7904 sec.
Iteration 9450 || Loss: 13.0793 || 10iter: 4.6583 sec.
Iteration 9460 || Lo

Iteration 10370 || Loss: 10.9125 || 10iter: 9.2947 sec.
Iteration 10380 || Loss: 12.3299 || 10iter: 6.2849 sec.
Iteration 10390 || Loss: 16.0326 || 10iter: 5.8865 sec.
Iteration 10400 || Loss: 15.3373 || 10iter: 6.1114 sec.
Iteration 10410 || Loss: 13.4173 || 10iter: 5.7901 sec.
Iteration 10420 || Loss: 18.7039 || 10iter: 4.3080 sec.
-------------
epoch 158 || Epoch_TRAIN_Loss:866.9069 ||Epoch_VAL_Loss:0.0000
timer:  42.5470 sec.
lr is: 0.0001
-------------
Epoch 159/200
-------------
train
Iteration 10430 || Loss: 15.0447 || 10iter: 4.8775 sec.
Iteration 10440 || Loss: 13.1233 || 10iter: 6.7415 sec.
Iteration 10450 || Loss: 12.2045 || 10iter: 5.6558 sec.
Iteration 10460 || Loss: 13.8782 || 10iter: 5.7125 sec.
Iteration 10470 || Loss: 14.1288 || 10iter: 5.1885 sec.
Iteration 10480 || Loss: 10.3342 || 10iter: 5.8033 sec.
Iteration 10490 || Loss: 13.2250 || 10iter: 4.8225 sec.
-------------
epoch 159 || Epoch_TRAIN_Loss:909.8899 ||Epoch_VAL_Loss:0.0000
timer:  41.0883 sec.
lr is: 0.0001


Iteration 11400 || Loss: 14.1533 || 10iter: 3.3881 sec.
Iteration 11410 || Loss: 15.9351 || 10iter: 3.1915 sec.
-------------
epoch 173 || Epoch_TRAIN_Loss:884.7244 ||Epoch_VAL_Loss:0.0000
timer:  25.6705 sec.
lr is: 0.0001
-------------
Epoch 174/200
-------------
train
Iteration 11420 || Loss: 15.8546 || 10iter: 3.0143 sec.
Iteration 11430 || Loss: 12.6643 || 10iter: 4.0291 sec.
Iteration 11440 || Loss: 12.7654 || 10iter: 3.3546 sec.
Iteration 11450 || Loss: 11.8436 || 10iter: 3.4508 sec.
Iteration 11460 || Loss: 10.7901 || 10iter: 3.4599 sec.
Iteration 11470 || Loss: 11.5479 || 10iter: 3.3270 sec.
Iteration 11480 || Loss: 12.6585 || 10iter: 3.1330 sec.
-------------
epoch 174 || Epoch_TRAIN_Loss:902.3531 ||Epoch_VAL_Loss:0.0000
timer:  25.4691 sec.
lr is: 0.0001
-------------
Epoch 175/200
-------------
train
Iteration 11490 || Loss: 12.3092 || 10iter: 5.0570 sec.
Iteration 11500 || Loss: 13.8599 || 10iter: 3.4007 sec.
Iteration 11510 || Loss: 13.5010 || 10iter: 3.4282 sec.
Iteratio

Iteration 12410 || Loss: 11.2111 || 10iter: 3.2243 sec.
Iteration 12420 || Loss: 12.0982 || 10iter: 3.7587 sec.
Iteration 12430 || Loss: 13.8475 || 10iter: 3.4758 sec.
Iteration 12440 || Loss: 14.8300 || 10iter: 3.6678 sec.
Iteration 12450 || Loss: 17.4080 || 10iter: 3.4875 sec.
Iteration 12460 || Loss: 12.6967 || 10iter: 3.3884 sec.
Iteration 12470 || Loss: 16.5735 || 10iter: 3.1351 sec.
-------------
epoch 189 || Epoch_TRAIN_Loss:912.0688 ||Epoch_VAL_Loss:0.0000
timer:  25.8486 sec.
lr is: 1e-05
-------------
Epoch 190/200
-------------
train
Iteration 12480 || Loss: 10.3492 || 10iter: 4.9027 sec.
Iteration 12490 || Loss: 15.6157 || 10iter: 3.6552 sec.
Iteration 12500 || Loss: 12.0285 || 10iter: 3.3948 sec.
Iteration 12510 || Loss: 13.4263 || 10iter: 3.4436 sec.
Iteration 12520 || Loss: 11.4418 || 10iter: 3.4434 sec.
Iteration 12530 || Loss: 12.8234 || 10iter: 3.2017 sec.
Iteration 12540 || Loss: 15.9670 || 10iter: 2.9974 sec.
-------------
val
-------------
epoch 190 || Epoch_TRAIN_

That's all :)